In [27]:
#!kill -9 -1
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)

In [29]:
# https://keras.io/
!pip install -q keras
import keras
#!apt --fix-broken install
#In a new virtual environment
#!pip install scipy==0.18.1
!pip install plotnine
from plotnine import ggplot

plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
    100% |████████████████████████████████| 42.5MB 842kB/s eta 0:00:01  7% |██▌                             | 3.3MB 11.7MB/s eta 0:00:04    15% |████▉                           | 6.4MB 6.5MB/s eta 0:00:06    16% |█████▎                          | 7.0MB 11.2MB/s eta 0:00:04    18% |█████▉                          | 7.8MB 6.3MB/s eta 0:00:06    24% |███████▉                        | 10.4MB 18.2MB/s eta 0:00:02    26% |████████▍                       | 11.1MB 9.5MB/s eta 0:00:04    41% |█████████████▍                  | 17.7MB 35.6MB/s eta 0:00:01         | 21.5MB 35.4MB/s eta 0:00:01    91% |█████████████████████████████▏  | 38.8MB 31.3MB/s eta 0:00:01
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.18.1 which is incompatible.
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
    100% |██████████████████

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [19]:
import os
import sys
import time

import keras.backend as K

import tensorflow as tf

from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation, Lambda, Reshape
from keras.layers.convolutional import Conv2D, Deconv2D, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

from keras.utils import generic_utils
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras.utils.generic_utils import Progbar


from IPython.core.debugger import set_trace

#!git clone https://github.com/siddharthalodha/mlsid.git
    
#!cp -r mlsid/models .
#!cp -r mlsid/figures .
#!cp -r mlsid/utils .

sys.path.append("./utils") # Utils
import general_utils
import data_utils

#!mkdir /figures

In [20]:
#Hyperparameters

backend="tensorflow"
dset="celebA"
generator="upsampling"
model_name="CNN"
batch_size=32
n_batch_per_epoch=200
nb_epoch=400
epoch=10
nb_classes=2
do_plot=True
bn_mode=2
img_dim=64
label_smoothing="store_true"
label_flipping=0
noise_scale=0.5
use_mbd="store_true"
image_data_format = "channels_last"

In [21]:
#Code to download CelebA hdf5 dataset from google drive
#!pip install PyDrive

#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

#fileId = drive.CreateFile({'id': '1cUBPxqU-9Y6f_OAfPwdRmPg7ruZ2sfdk'})
#print(fileId['title'])  # CelebA Dataset
#fileId.GetContentFile('CelebA_64_data.h5')  # Save Drive file as a local file

#fileId = drive.CreateFile({'id': '10B5fIjTwgk1IyXNGbWTzYEXMxnbFixTJ'})
#print(fileId['title'])  # CelebA Dataset
#fileId.GetContentFile('MNIST.h5')  # Save Drive file as a local file


In [22]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
      raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

Found GPU at: /device:GPU:0


In [23]:
def generator_upsampling(noise_dim, img_dim, bn_mode, model_name="generator_upsampling", dset="celebA"):
    """
    Generator model of the DCGAN
    args : img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    s = img_dim[1]
    f = 512

    if dset == "mnist":
        start_dim = int(s / 4)
        nb_upconv = 2
    else:
        start_dim = int(s / 16)
        nb_upconv = 4

    if K.image_data_format() == "channels_first":
        bn_axis = 1
        reshape_shape = (f, start_dim, start_dim)
        output_channels = img_dim[0]
    else:
        reshape_shape = (start_dim, start_dim, f)
        bn_axis = -1
        output_channels = img_dim[-1]

    gen_input = Input(shape=noise_dim, name="generator_input")

    x = Dense(f * start_dim * start_dim, input_dim=noise_dim)(gen_input)
    x = Reshape(reshape_shape)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation("relu")(x)

    # Upscaling blocks
    for i in range(nb_upconv):
        x = UpSampling2D(size=(2, 2))(x)
        nb_filters = int(f / (2 ** (i + 1)))
        x = Conv2D(nb_filters, (3, 3), padding="same")(x)
        x = BatchNormalization(axis=1)(x)
        x = Activation("relu")(x)
        x = Conv2D(nb_filters, (3, 3), padding="same")(x)
        x = Activation("relu")(x)

    x = Conv2D(output_channels, (3, 3), name="gen_Conv2D_final", padding="same", activation='tanh')(x)

    generator_model = Model(inputs=[gen_input], outputs=[x], name=model_name)

    return generator_model


def generator_deconv(noise_dim, img_dim, bn_mode, batch_size, model_name="generator_deconv", dset="mnist"):
    """
    Generator model of the DCGAN
    args : nb_classes (int) number of classes
           img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    assert K.backend() == "tensorflow", "Deconv not implemented with theano"

    s = img_dim[1]
    f = 512

    if dset == "mnist":
        start_dim = int(s / 4)
        nb_upconv = 2
    else:
        start_dim = int(s / 16)
        nb_upconv = 4

    reshape_shape = (start_dim, start_dim, f)
    bn_axis = -1
    output_channels = img_dim[-1]

    gen_input = Input(shape=noise_dim, name="generator_input")

    x = Dense(f * start_dim * start_dim, input_dim=noise_dim)(gen_input)
    x = Reshape(reshape_shape)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation("relu")(x)

    # Transposed conv blocks
    for i in range(nb_upconv - 1):
        nb_filters = int(f / (2 ** (i + 1)))
        s = start_dim * (2 ** (i + 1))
        o_shape = (batch_size, s, s, nb_filters)
        x = Deconv2D(nb_filters, (3, 3), output_shape=o_shape, strides=(2, 2), padding="same")(x)
        x = BatchNormalization(axis=-1)(x)
        x = Activation("relu")(x)

    # Last block
    s = start_dim * (2 ** (nb_upconv))
    o_shape = (batch_size, s, s, output_channels)
    x = Deconv2D(output_channels, (3, 3), output_shape=o_shape, strides=(2, 2), padding="same")(x)
    x = Activation("tanh")(x)

    generator_model = Model(inputs=[gen_input], outputs=[x], name=model_name)

    return generator_model


def DCGAN_discriminator(noise_dim, img_dim, bn_mode, model_name="DCGAN_discriminator", dset="celebA", use_mbd=False):
    """
    Discriminator model of the DCGAN
    args : img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    if K.image_data_format() == "channels_first":
        bn_axis = 1
    else:
        bn_axis = -1

    disc_input = Input(shape=img_dim, name="discriminator_input")

    if dset == "mnist":
        list_f = [128]

    else:
        list_f = [64, 128, 256]

    # First conv
    x = Conv2D(32, (3, 3), strides=(2, 2), name="disc_Conv2D_1", padding="same")(disc_input)
    x = BatchNormalization(axis=bn_axis)(x)
    x = LeakyReLU(0.2)(x)

    # Next convs
    for i, f in enumerate(list_f):
        name = "disc_Conv2D_%s" % (i + 2)
        x = Conv2D(f, (3, 3), strides=(2, 2), name=name, padding="same")(x)
        x = BatchNormalization(axis=bn_axis)(x)
        x = LeakyReLU(0.2)(x)

    x = Flatten()(x)

    def minb_disc(x):
        diffs = K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, [1, 2, 0]), 0)
        abs_diffs = K.sum(K.abs(diffs), 2)
        x = K.sum(K.exp(-abs_diffs), 2)

        return x

    def lambda_output(input_shape):
        return input_shape[:2]

    num_kernels = 100
    dim_per_kernel = 5

    M = Dense(num_kernels * dim_per_kernel, use_bias=False, activation=None)
    MBD = Lambda(minb_disc, output_shape=lambda_output)

    if use_mbd:
        x_mbd = M(x)
        x_mbd = Reshape((num_kernels, dim_per_kernel))(x_mbd)
        x_mbd = MBD(x_mbd)
        x = Concatenate(axis=bn_axis)([x, x_mbd])

    x = Dense(2, activation='softmax', name="disc_dense_2")(x)

    discriminator_model = Model(inputs=[disc_input], outputs=[x], name=model_name)

    return discriminator_model


def DCGAN(generator, discriminator_model, noise_dim, img_dim):

    noise_input = Input(shape=noise_dim, name="noise_input")

    generated_image = generator(noise_input)
    DCGAN_output = discriminator_model(generated_image)

    DCGAN = Model(inputs=[noise_input],
                  outputs=[DCGAN_output],
                  name="DCGAN")
    return DCGAN


def load(model_name, noise_dim, img_dim, bn_mode, batch_size, dset="mnist", use_mbd=False):

    if model_name == "generator_upsampling":
        model = generator_upsampling(noise_dim, img_dim, bn_mode, model_name=model_name, dset=dset)
        model.summary()
        #plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model
    if model_name == "generator_deconv":
        model = generator_deconv(noise_dim, img_dim, bn_mode, batch_size, model_name=model_name, dset=dset)
        model.summary()
        #plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model
    if model_name == "DCGAN_discriminator":
        model = DCGAN_discriminator(noise_dim, img_dim, bn_mode, model_name=model_name, dset=dset, use_mbd=use_mbd)
        model.summary()
        #plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model

In [24]:
def train_GAN(batch_size,n_batch_per_epoch,nb_epoch,generator,model_name,image_data_format,img_dim,bn_mode,label_smoothing,label_flipping,noise_scale,dset,use_mbd,epoch_size):
    # Setup environment (logging directory etc)
    #general_utils.setup_logging(model_name) #change this if setup_logging moved to separate file

    # Load and rescale data
    if dset == "celebA":
        X_real_train = data_utils.load_celebA(img_dim, image_data_format) #change func call if file moved
    if dset == "mnist":
        X_real_train, _, _, _ = data_utils.load_mnist(image_data_format) #change func call if file moved
    img_dim = X_real_train.shape[-3:]
    noise_dim = (100,)
    
    try:

        # Create optimizers
        opt_dcgan = Adam(lr=1E-3, beta_1=0.5, beta_2=0.999, epsilon=1e-08)
        opt_discriminator = SGD(lr=1E-3, momentum=0.9, nesterov=True)

        # Load generator model
        generator_model = load("generator_%s" % generator,
                                      noise_dim,
                                      img_dim,
                                      bn_mode,
                                      batch_size,
                                      dset=dset,
                                      use_mbd=use_mbd)
        
        # Load discriminator model
        discriminator_model = load("DCGAN_discriminator",
                                          noise_dim,
                                          img_dim,
                                          bn_mode,
                                          batch_size,
                                          dset=dset,
                                          use_mbd=use_mbd)

        generator_model.compile(loss='mse', optimizer=opt_discriminator)
        discriminator_model.trainable = False
        DCGAN_model = DCGAN(generator_model,
                                   discriminator_model,
                                   noise_dim,
                                   img_dim)
        
        loss = ['binary_crossentropy']
        loss_weights = [1]
        DCGAN_model.compile(loss=loss, loss_weights=loss_weights, optimizer=opt_dcgan)

        discriminator_model.trainable = True
        discriminator_model.compile(loss='binary_crossentropy', optimizer=opt_discriminator)

        gen_loss = 100
        disc_loss = 100

        # Start training
        print("Start training")
        for e in range(nb_epoch):
            # Initialize progbar and batch counter
            #progbar = Progbar(epoch_size)
            batch_counter = 1
            start = time.time()
            for X_real_batch in data_utils.gen_batch(X_real_train, batch_size):

                # Create a batch to feed the discriminator model
                X_disc, y_disc = data_utils.get_disc_batch(X_real_batch,
                                                           generator_model,
                                                           batch_counter,
                                                           batch_size,
                                                           noise_dim,
                                                           noise_scale=noise_scale,
                                                           label_smoothing=label_smoothing,
                                                           label_flipping=label_flipping)

                # Update the discriminator
                disc_loss = discriminator_model.train_on_batch(X_disc, y_disc)

                # Create a batch to feed the generator model
                X_gen, y_gen = data_utils.get_gen_batch(batch_size, noise_dim, noise_scale=noise_scale)

                # Freeze the discriminator
                discriminator_model.trainable = False
                gen_loss = DCGAN_model.train_on_batch(X_gen, y_gen)
                # Unfreeze the discriminator
                discriminator_model.trainable = True

                batch_counter += 1
                #progbar.add(batch_size, values=[("D logloss", disc_loss),
                #                                ("G logloss", gen_loss)])

                 #Save images for visualization
                if batch_counter % 100 == 0:
                     data_utils.plot_generated_batch(X_real_batch, generator_model,
                                                    batch_size, noise_dim, image_data_format)
                    
                if batch_counter >= n_batch_per_epoch:
                    break

            print("")
            print('Epoch %s/%s, Time: %s' % (e + 1, nb_epoch, time.time() - start))

            if e % 5 == 0:
                gen_weights_path = os.path.join('./models/%s/gen_weights_epoch%s.h5' % (model_name, e))
                generator_model.save_weights(gen_weights_path, overwrite=True)

                disc_weights_path = os.path.join('./models/%s/disc_weights_epoch%s.h5' % (model_name, e))
                discriminator_model.save_weights(disc_weights_path, overwrite=True)

                DCGAN_weights_path = os.path.join('./models/%s/DCGAN_weights_epoch%s.h5' % (model_name, e))
                DCGAN_model.save_weights(DCGAN_weights_path, overwrite=True)
    except Exception as e:
            print(e)

In [25]:
os.environ["KERAS_BACKEND"] = "tensorflow"
image_data_format = "channels_last"
K.set_image_data_format(image_data_format)    
# Launch training
train_GAN(batch_size,n_batch_per_epoch,nb_epoch,generator,model_name,image_data_format,img_dim,bn_mode,label_smoothing,label_flipping,noise_scale,dset,use_mbd,epoch) #change func if file is moved

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_input (InputLayer) (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_3 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 4, 4, 512)         2048      
_________________________________________________________________
activation_10 (Activation)   (None, 4, 4, 512)         0         
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 256)         1179904   
__________

Test2
Start training

Epoch 1/400, Time: 37.4510235786438

Epoch 2/400, Time: 28.17511487007141

Epoch 3/400, Time: 32.708229303359985

Epoch 4/400, Time: 31.214498281478882

Epoch 5/400, Time: 30.54066276550293

Epoch 6/400, Time: 32.50483560562134

Epoch 7/400, Time: 30.120712518692017

Epoch 8/400, Time: 30.980198860168457

Epoch 9/400, Time: 32.25708341598511

Epoch 10/400, Time: 27.368842601776123

Epoch 11/400, Time: 26.32403016090393

Epoch 12/400, Time: 28.45842146873474

Epoch 13/400, Time: 31.983362674713135

Epoch 14/400, Time: 31.154237985610962

Epoch 15/400, Time: 28.175098180770874

Epoch 16/400, Time: 30.654862880706787

Epoch 17/400, Time: 32.28744864463806

Epoch 18/400, Time: 32.60296964645386

Epoch 19/400, Time: 33.17178225517273

Epoch 20/400, Time: 27.40247368812561

Epoch 21/400, Time: 26.49756932258606

Epoch 22/400, Time: 30.512654542922974

Epoch 23/400, Time: 34.331533670425415

Epoch 24/400, Time: 29.20116877555847

Epoch 25/400, Time: 32.51710629463196

Ep


Epoch 206/400, Time: 32.15684795379639

Epoch 207/400, Time: 31.330183029174805

Epoch 208/400, Time: 27.76236629486084

Epoch 209/400, Time: 30.698052167892456

Epoch 210/400, Time: 30.25640344619751

Epoch 211/400, Time: 30.488356113433838

Epoch 212/400, Time: 31.998400449752808

Epoch 213/400, Time: 33.5871844291687

Epoch 214/400, Time: 30.52629852294922

Epoch 215/400, Time: 33.018577098846436

Epoch 216/400, Time: 31.96224880218506

Epoch 217/400, Time: 29.577242136001587

Epoch 218/400, Time: 34.169039726257324

Epoch 219/400, Time: 29.455947637557983

Epoch 220/400, Time: 27.27121877670288

Epoch 221/400, Time: 29.692992210388184

Epoch 222/400, Time: 31.523547887802124

Epoch 223/400, Time: 32.92514967918396

Epoch 224/400, Time: 31.45634698867798

Epoch 225/400, Time: 31.857252836227417

Epoch 226/400, Time: 28.37203598022461

Epoch 227/400, Time: 27.000257968902588

Epoch 228/400, Time: 31.220900774002075

Epoch 229/400, Time: 34.34600043296814

Epoch 230/400, Time: 28.479

In [26]:
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)